# Optimization Modes

This notebook demonstrates different optimization approaches in **flixopt**:

1. **Full Optimization** - Solve the complete problem at once
2. **Resampled Optimization** - Reduce resolution for faster solving
3. **Clustered Optimization** - Reduce complexity via time series aggregation

These modes offer trade-offs between solution quality and computation time, essential for large-scale problems.

## Setup

In [ ]:
import pathlib

import pandas as pd
import xarray as xr
from IPython.display import IFrame

import flixopt as fx

fx.CONFIG.notebook()

## Configuration

Select which optimization modes to run:

In [ ]:
# Resampled optimization parameters
resample_freq = '2h'  # Downsample to 2-hour resolution

# Clustered optimization parameters
clustering_parameters = fx.ClusteringParameters(
    hours_per_period=6,
    nr_of_periods=4,
    fix_storage_flows=False,
    aggregate_data_and_fix_non_binary_vars=True,
    percentage_of_period_freedom=0,
    penalty_of_period_freedom=0,
)

imbalance_penalty = 1e5

## Load Data

Import time series data for one week:

In [ ]:
# Load data from CSV
data_path = pathlib.Path('..') / '..' / 'examples' / 'resources' / 'Zeitreihen2020.csv'
data_import = pd.read_csv(data_path, index_col=0).sort_index()

# Filter to one week
filtered_data = data_import['2020-01-01':'2020-01-07 23:45:00']
filtered_data.index = pd.to_datetime(filtered_data.index)
timesteps = filtered_data.index

print(f'Time range: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

In [ ]:
# Extract columns
electricity_demand = filtered_data['P_Netz/MW'].to_numpy()
heat_demand = filtered_data['Q_Netz/MW'].to_numpy()
electricity_price = filtered_data['Strompr.€/MWh'].to_numpy()
gas_price = filtered_data['Gaspr.€/MWh'].to_numpy()

## Create TimeSeriesData Objects

`TimeSeriesData` allows specifying clustering weights and groups for aggregation:

In [ ]:
TS_heat_demand = fx.TimeSeriesData(heat_demand)
TS_electricity_demand = fx.TimeSeriesData(electricity_demand, clustering_weight=0.7)
TS_electricity_price_sell = fx.TimeSeriesData(-(electricity_price - 0.5), clustering_group='p_el')
TS_electricity_price_buy = fx.TimeSeriesData(electricity_price + 0.5, clustering_group='p_el')

## Build FlowSystem

We define a function to create the FlowSystem so we can easily create fresh copies:

In [ ]:
flow_system = fx.FlowSystem(timesteps)

# Add buses
flow_system.add_elements(
    fx.Bus('Strom', carrier='electricity', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Fernwärme', carrier='heat', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Gas', carrier='gas', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Kohle', carrier='fuel', imbalance_penalty_per_flow_hour=imbalance_penalty),
)

# Add effects
costs = fx.Effect('costs', '€', 'Kosten', is_standard=True, is_objective=True)
CO2 = fx.Effect('CO2', 'kg', 'CO2_e-Emissionen')
PE = fx.Effect('PE', 'kWh_PE', 'Primärenergie')

# Gas Boiler
gaskessel = fx.linear_converters.Boiler(
    'Kessel',
    thermal_efficiency=0.85,
    thermal_flow=fx.Flow(label='Q_th', bus='Fernwärme'),
    fuel_flow=fx.Flow(
        label='Q_fu',
        bus='Gas',
        size=95,
        relative_minimum=12 / 95,
        previous_flow_rate=20,
        status_parameters=fx.StatusParameters(effects_per_startup=1000),
    ),
)

# Coal CHP
kwk = fx.linear_converters.CHP(
    'BHKW2',
    thermal_efficiency=0.58,
    electrical_efficiency=0.22,
    status_parameters=fx.StatusParameters(effects_per_startup=24000),
    electrical_flow=fx.Flow('P_el', bus='Strom', size=200),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme', size=200),
    fuel_flow=fx.Flow('Q_fu', bus='Kohle', size=288, relative_minimum=87 / 288, previous_flow_rate=100),
)

# Thermal Storage
speicher = fx.Storage(
    'Speicher',
    capacity_in_flow_hours=684,
    initial_charge_state=137,
    minimal_final_charge_state=137,
    maximal_final_charge_state=158,
    eta_charge=1,
    eta_discharge=1,
    relative_loss_per_hour=0.001,
    prevent_simultaneous_charge_and_discharge=True,
    charging=fx.Flow('Q_th_load', size=137, bus='Fernwärme'),
    discharging=fx.Flow('Q_th_unload', size=158, bus='Fernwärme'),
)

# Sinks and Sources
waermelast = fx.Sink(
    'Wärmelast', inputs=[fx.Flow('Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=TS_heat_demand)]
)
strom_last = fx.Sink(
    'Stromlast', inputs=[fx.Flow('P_el_Last', bus='Strom', size=1, fixed_relative_profile=TS_electricity_demand)]
)
gas_tarif = fx.Source(
    'Gastarif',
    outputs=[fx.Flow('Q_Gas', bus='Gas', size=1000, effects_per_flow_hour={costs.label: gas_price, CO2.label: 0.3})],
)
kohle_tarif = fx.Source(
    'Kohletarif',
    outputs=[fx.Flow('Q_Kohle', bus='Kohle', size=1000, effects_per_flow_hour={costs.label: 4.6, CO2.label: 0.3})],
)
strom_einspeisung = fx.Sink(
    'Einspeisung', inputs=[fx.Flow('P_el', bus='Strom', size=1000, effects_per_flow_hour=TS_electricity_price_sell)]
)
strom_tarif = fx.Source(
    'Stromtarif',
    outputs=[
        fx.Flow(
            'P_el',
            bus='Strom',
            size=1000,
            effects_per_flow_hour={costs.label: TS_electricity_price_buy, CO2.label: 0.3},
        )
    ],
)

# Assemble FlowSystem
flow_system.add_elements(costs, CO2, PE)
flow_system.add_elements(
    gaskessel,
    waermelast,
    strom_last,
    gas_tarif,
    kohle_tarif,
    strom_einspeisung,
    strom_tarif,
    kwk,
    speicher,
)

# Visualize
flow_system.topology.plot('topology.html', show=False)
IFrame('topology.html', width='100%', height=400)

## Run Optimizations

We'll run three different optimization modes and compare their results.

### 1. Full Optimization

Solve the entire problem at once with full time resolution:

In [ ]:
results = {}  # Store results for comparison
solver = fx.solvers.HighsSolver(mip_gap=0.01 / 100, time_limit_seconds=60)

fs_full = flow_system.copy()
fs_full.optimize(solver)
results['Full'] = fs_full
print(f'Full optimization: {len(fs_full.timesteps)} timesteps')
print(f'Total costs: {fs_full.solution["costs"].item():.2f} €')

### 2. Resampled Optimization

Reduce time resolution using `transform.resample()`. Useful for:
- Faster computation with acceptable accuracy loss
- Quick screening of design alternatives

In [ ]:
fs_resampled = flow_system.transform.resample(resample_freq)
fs_resampled.optimize(solver)
results['Resampled'] = fs_resampled
print(f'Resampled optimization: {len(fs_resampled.timesteps)} timesteps (from {len(flow_system.timesteps)})')
print(f'Total costs: {fs_resampled.solution["costs"].item():.2f} €')

### 3. Clustered Optimization

Reduce time series complexity through clustering using `transform.cluster()`. Useful for:
- Investment planning studies
- Screening large numbers of scenarios
- Problems where computational time is critical

In [ ]:
# Add extreme period detection
clustering_parameters.time_series_for_high_peaks = [TS_heat_demand]
clustering_parameters.time_series_for_low_peaks = [TS_electricity_demand, TS_heat_demand]

fs_clustered = flow_system.transform.cluster(clustering_parameters)
fs_clustered.optimize(solver)
results['Clustered'] = fs_clustered
print(f'Clustered optimization: {clustering_parameters.nr_of_periods} representative periods')
print(f'Total costs: {fs_clustered.solution["costs"].item():.2f} €')

## Compare Results

### Total Costs Comparison

In [ ]:
cost_comparison = pd.DataFrame(
    {
        name: {
            'Total Costs [€]': fs.solution['costs'].item(),
            'Timesteps': len(fs.timesteps),
        }
        for name, fs in results.items()
    }
).T

# Add percentage difference from full
full_cost = results['Full'].solution['costs'].item()
cost_comparison['Diff from Full [%]'] = ((cost_comparison['Total Costs [€]'] - full_cost) / full_cost * 100).round(2)

print(cost_comparison)

### Storage Charge State Comparison

In [ ]:
# Combine charge states from all modes. ffill for reduced time index when resampled
charge_states = xr.Dataset(
    {
        name: fs.solution['Speicher|charge_state']
        for name, fs in results.items()
        if 'Speicher|charge_state' in fs.solution
    }
).ffill('time')

fx.plotting.with_plotly(
    charge_states,
    mode='line',
    title='Storage Charge State Comparison',
    ylabel='Charge state [kWh]',
    xlabel='Time',
).show()

### CHP Operation Comparison

In [ ]:
# Combine CHP thermal output from all modes
chp_output = xr.Dataset(
    {
        name: fs.solution['BHKW2(Q_th)|flow_rate']
        for name, fs in results.items()
        if 'BHKW2(Q_th)|flow_rate' in fs.solution
    }
).ffill('time')

fx.plotting.with_plotly(
    chp_output,
    mode='line',
    title='CHP Thermal Output Comparison',
    ylabel='Flow rate [MW]',
    xlabel='Time',
).show()

### Detailed Results for Full Optimization

In [ ]:
results['Full'].statistics.plot.balance('Fernwärme')

In [ ]:
results['Full'].statistics.plot.heatmap('Speicher')

## Summary

| Mode | API | Best For | Trade-off |
|------|-----|----------|----------|
| **Full** | `flow_system.optimize(solver)` | Small-medium problems, exact solutions | Memory/time limited |
| **Resampled** | `flow_system.transform.resample('2h').optimize(solver)` | Quick estimates, screening | Loses temporal detail |
| **Clustered** | `flow_system.transform.cluster(params).optimize(solver)` | Investment planning, long horizons | Approximates temporal dynamics |

Choose the mode based on your problem size and accuracy requirements!